# Introduction

1. add the zeros (n-1)
2. take top n
2b. then add in a random number col, then order by id then random number, then drop random number (ergo they are shuffled)
3. then values
4. then split into number of ids (works because they're the same size)
5. then reshape to #dfs (ids), #of dims https://stackoverflow.com/questions/9057379/correct-and-efficient-way-to-flatten-array

6. add support for columns that are used for all

In [99]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle

First up, let's create our dummy data, nto terribly efficient, but not a disaster either

In [209]:
df = pd.DataFrame(columns = ['id','1','2','3','4','5','6','7','8','10'])
my_randoms = []
for i in range(2,10):
    my_randoms+=[random.randrange(1,100000,1) for _ in range (10000)] 
df['id'] = my_randoms
for i in range(1,11):
    df[str(i)] = i
    
print("Number of unique ids", len(df['id'].unique()))

Number of unique ids 55109


Ok, 55,000 odd records, close enough to the 200k ids (and 2m contacts) that the business problem had

## Creating the functions

Back in the day I did this with a loop over the DataFrame, most definitely unvectorised...  Still it worked, and on some decent hardware it was ahem, tolerable.  Only ever menat to be a PoC it was a good example of bad code migrating to (sort-of) production).


In [216]:
def pad_df(df,pad_value,pad_length,id_col):
    id_list = [[id_]*pad_length for id_ in df[id_col].unique()]
    id_list = [y for x in id_list for y in x]
    df_tmp = pd.DataFrame(columns = df.columns)
    df_tmp[id_col] = id_list
    df_tmp = df_tmp.fillna(pad_value)
    df = df.append(df_tmp)
    df = shuffle(df)
    df = df.sort_values(by=id_col)
    df = df.groupby('id').head(pad_length)
    return df

def df_padded_to_X(df,id_col):
    id_count = df[id_col].unique()
    X_tmp1 = np.array(df_padded.drop('id',axis=1).values)
    X_tmp2 = np.ravel(X_tmp1)
    X = np.array(np.array_split(X_tmp1,id_count))
    return X

Already things feel better, with RADICALLY reduced line count and a total absence of looping over lists (unless you count the list comprehensions, which are likely to be the new chokepoints). Let's time things!

First up, the creation of the padded dataframe...

In [213]:
%timeit df_padded = pad_df(df,0,10,'id')

4.39 s ± 47.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


4 and a bit seconds is quite frankly blisteringly quick compared to the hours that the old code took to run.  I'll consider this a win, but what about the data into an X format?

In [212]:
%timeit X = df_padded_to_X(df_padded,'id')

1.4 ms ± 49.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


And what do you know?  Trivial time with the heavy use of numpy's optimised code a clear winner (mental notes on the data frame manipualtions above...)

So what have we learnt?  Well, first up we've learnt that being Pythonic really pays off, this code is both shorter in lines, clearner to read (probably) and several orders of magnitude faster.  I'll consider this a win, and wish I'd the headspace to do this back in the day.  Still, one less simian on the back...

Next up:

- add in unit tests
- include one hot encoding coverage
- docstrings for the functions
- build into a usable pipeline

In [ ]:
#add in time it
# add in unit tests
# add in functionality to include single id values (e.g. disocunt contact)
# add docsstrings etc.